#### 图像拼接（参考代码）

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# 读取图片
img_p1 = cv2.imread("./photos/ori_left.jpg")
img_p2 = cv2.imread("./photos/ori_right_mirror.jpg")

# 转为灰度图
# Q：为什么要转化为灰度图？
gimg_p1 = cv2.cvtColor(img_p1, cv2.COLOR_BGR2GRAY)
gimg_p2 = cv2.cvtColor(img_p2, cv2.COLOR_BGR2GRAY)

# 初始化SIFT检测器并提取特征
sift = cv2.SIFT_create()
kp_p1, des_p1 = sift.detectAndCompute(gimg_p1, None)  # 图像p1的关键点和描述符
kp_p2, des_p2 = sift.detectAndCompute(gimg_p2, None)  # 图像p2的关键点和描述符

# 使用FLANN进行特征匹配
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=100)  # 检查次数越多，匹配越准确但速度越慢
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des_p1, des_p2, k=2)  # k=2表示每个特征点返回2个最佳匹配


In [ ]:
# 应用Lowe's比率测试筛选优质匹配点
good_matches = []
for m, n in matches:
    if m.distance < 0.7 * n.distance:  # 比率阈值通常取0.7-0.8
        good_matches.append(m)

# 绘制匹配的SIFT特征点
matched_keypoints_img = cv2.drawMatches(
    img_p1, kp_p1, img_p2, kp_p2, good_matches,
    None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)

# 提取匹配点的坐标
src_pts = np.float32([kp_p2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
dst_pts = np.float32([kp_p1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)

# 使用RANSAC算法估计单应矩阵
H, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)


In [ ]:
# 获取图像输入尺寸
h_p1, w_p1 = img_p1.shape[:2]
h_p2, w_p2 = img_p2.shape[:2]

# 计算图像p2变换后的四个角点坐标
pts = np.float32([[0, 0], [0, h_p2], [w_p2, h_p2], [w_p2, 0]]).reshape(-1, 1, 2)
dst_corners = cv2.perspectiveTransform(pts, H)

# 确定拼接后图像的最终尺寸(包含所有像素)
all_corners = np.concatenate([dst_corners,
                              np.float32([[0, 0], [w_p1, 0], [w_p1, h_p1], [0, h_p1]]).reshape(-1, 1, 2)],
                             axis=0)
[x_min, y_min] = np.int32(all_corners.min(axis=0).ravel() - 0.5)
[x_max, y_max] = np.int32(all_corners.max(axis=0).ravel() + 0.5)

# 创建平移矩阵
translation_matrix = np.array([[1, 0, -x_min], [0, 1, -y_min], [0, 0, 1]], dtype=np.float32)

# 对图像p2应用透视变换并进行平移
fus_img = cv2.warpPerspective(
    img_p2,
    translation_matrix @ H,  # 组合平移矩阵与单应矩阵
    (x_max - x_min, y_max - y_min)  # 输出图像尺寸
)

# 将图像p1复制到拼接图像的正确位置
fus_img[-y_min:h_p1 - y_min, -x_min:w_p1 - x_min] = img_p1


In [ ]:
# 显示匹配关键点及拼接结果
plt.figure(figsize=(80, 20))
plt.subplot(4, 1, 1)
plt.imshow(cv2.cvtColor(img_p1, cv2.COLOR_BGR2RGB))
plt.title("Original Image 1")

plt.subplot(4, 1, 2)
plt.imshow(cv2.cvtColor(img_p2, cv2.COLOR_BGR2RGB))
plt.title("Original Image 2")

plt.subplot(4, 1, 3)
plt.imshow(cv2.cvtColor(matched_keypoints_img, cv2.COLOR_BGR2RGB))
plt.title("Matched Keypoints")

plt.subplot(4, 1, 4)
plt.imshow(cv2.cvtColor(fus_img, cv2.COLOR_BGR2RGB))
plt.title("Stitched Image")

cv2.imwrite("baseline_road_image.jpg", fus_img)
